In [1]:
from Dental_Tool.Grad_CAM import *
from Dental_Tool.Process_results import *
from scipy.ndimage.interpolation import zoom
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K 
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import keras
import time
import cv2
import os

Using TensorFlow backend.


In [2]:
root_dir = "Results/Emsemble_5_Fold______interdental20200909-170725_Max"

In [3]:
for i in range(1, 6, 1):
        K.clear_session()
        dataset_table = f"{root_dir}/Fold_{i}/table/test_dataset.csv"
        param_path = f"{root_dir}/Fold_{i}/Inception_Fold_{i}/parameter"
        grad_cam_dir = f"{root_dir}/Fold_{i}/Inception_Fold_{i}/Grad_Cam"
        
        if not os.path.isdir(grad_cam_dir) : os.makedirs(grad_cam_dir)
        best_param = get_k_top_value(param_path, k_th=1)
        
        K.clear_session()
        print("Loading model......")
        start = time.time()
        model = load_model(best_param[0])
        end   = time.time()
        elapse = end - start
        print(f"Finish Loading, cost {elapse} sec.....")
        
        test_data = pd.read_csv(dataset_table)
        all_GT = list(test_data["Class"])
        all_image_path = list(test_data["Path"])
        count = 0
        
        for path, gt in zip(all_image_path, all_GT):
                if count >= 200: break 
                basename = os.path.basename(path)
                
                test_image = cv2.imread(path, 0)
                test_image = cv2.resize(test_image,(180, 200))
                test_image = np.reshape(test_image, (*test_image.shape, 1))
                test_image = np.expand_dims(test_image, axis=0)
                prediction = model.predict(test_image)
                predict    = np.argmax(prediction[0])
                gradcam    = grad_cam(model,     test_image, layer_name='mixed10', H=180, W=200)
                
        #         gradcamplus = grad_cam_plus(model,test_image, layer_name='mixed10', H=200, W=180)

                orig_img = cv2.imread(path)

                gradcam = cv2.resize(gradcam, (orig_img.shape[1], orig_img.shape[0]))

#                 fig, ax = plt.subplots(nrows=1,ncols=2)
#                 plt.subplot(121)
#                 plt.imshow(orig_img, cmap='gray')
#                 plt.title("input image")
#                 plt.axis('off')


#                 plt.subplot(122)
                plt.imshow(orig_img, cmap='gray')
                plt.imshow(gradcam,alpha=0.5,cmap="jet")
                plt.axis('off')
            
                image_name = os.path.splitext(basename)[0]
                plt.savefig(f"{grad_cam_dir}/{image_name}_GT_{gt}_PD_{predict}.png")
                plt.close()
                count += 1
        K.clear_session()
#                 plt.title("Grad-CAM")
#                 plt.show()

Loading model......
Finish Loading, cost 443.6230425834656 sec.....
Loading model......
Finish Loading, cost 456.5666937828064 sec.....
Loading model......
Finish Loading, cost 453.281183719635 sec.....
Loading model......
Finish Loading, cost 460.01553773880005 sec.....
Loading model......
Finish Loading, cost 467.0210380554199 sec.....


In [4]:
# path = all_image_path[0]
# test_image = cv2.imread(path, 0)
# test_image = cv2.resize(test_image,(180, 200))
# test_image = np.reshape(test_image, (*test_image.shape, 1))
# test_image = np.expand_dims(test_image, axis=0)

# predictions = model.predict(test_image)
# predictions

In [5]:
# for path in all_image_path:
# #         path = all_image_path[0]
#         test_image = cv2.imread(path, 0)
#         test_image = cv2.resize(test_image,(180, 200))
#         test_image = np.reshape(test_image, (*test_image.shape, 1))
#         test_image = np.expand_dims(test_image, axis=0)

#         gradcam     = grad_cam(model,     test_image, layer_name='mixed10', H=180, W=200)
# #         gradcamplus = grad_cam_plus(model,test_image, layer_name='mixed10', H=200, W=180)

#         orig_img = cv2.imread(path)
        
#         gradcam = cv2.resize(gradcam, (orig_img.shape[1], orig_img.shape[0]))
                                          
#         # print("class activation map for:",top[0])
#         fig, ax = plt.subplots(nrows=1,ncols=2)
#         plt.subplot(121)
#         plt.imshow(orig_img, cmap='gray')
#         plt.title("input image")
#         plt.axis('off')


#         plt.subplot(122)
#         plt.imshow(orig_img, cmap='gray')
#         plt.imshow(gradcam,alpha=0.4,cmap="jet")
#         plt.axis('off')
#         plt.title("Grad-CAM")
#         plt.show()

#         plt.subplot(133)
#         plt.imshow(orig_img, cmap='gray')
#         plt.imshow(gradcamplus,alpha=0.4,cmap="jet")
#         plt.title("Grad-CAM++")
#         plt.axis('off')
#         plt.show()